### Importing packages

In [1]:
# Libraries
import time

# Reading in files
import pandas as pd
import numpy as np

# Progress bar
from tqdm.auto import tqdm

# Torch modules
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup

# Lightning modules
import pytorch_lightning as pl
from torchmetrics.functional import accuracy, auroc
from torchmetrics import F1Score
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall

# Split dataset/validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import StratifiedKFold

# Importing own functions 
from extract.importing_data import get_section

C:\Users\Benjamin Aw\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Reading in data

In [2]:
df = pd.read_csv("../Data/Collated_dataset_for_scientific_papers.csv")

In [3]:
df = df[["Intro Concl", "Labels"]]

In [4]:
df.columns = ["string", "label"]

In [5]:
possible_labels = list(df['label'].unique())
possible_labels_num = list(range(0,len(possible_labels)))

In [6]:
len(possible_labels)

21

### Tokenizer 

In [7]:
MODEL_NAME = 'allenai/specter'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [8]:
mapping = zip(possible_labels, possible_labels_num)
label_to_idx = {label: num  for label, num in mapping}
mapping = zip(possible_labels, possible_labels_num)
idx_to_label = {num: label for label, num in mapping}

In [9]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 5
BATCH_SIZE = 8 # Changes: Edit the batch size here
KFOLD = 5

In [10]:
class SpecterDataset(Dataset):
    
    def __init__(self, data: pd.DataFrame, tokenizer: AutoTokenizer, max_token_len: int = MAX_TOKEN_COUNT, mapping = label_to_idx):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
        self.mapping = mapping
    
    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index: int):
    
        data_row = self.data.iloc[index]
        text = data_row["string"]
        labels = self.mapping[data_row["label"]]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_token_len,
          return_token_type_ids=False,
          padding="max_length",
          truncation=True,
          return_attention_mask=True,
          return_tensors='pt',
        )

        return dict(
          text=text,
          input_ids=encoding["input_ids"].flatten(),
          attention_mask=encoding["attention_mask"].flatten(),
          labels=labels
        )

In [11]:
class SpecterDataModule(pl.LightningDataModule):
    
    def __init__(self, df,
                 tokenizer,
                 k = 0,  # fold number
                 split_seed = 123,  # split needs to be always the same for correct cross validation
                 num_splits = KFOLD,
                 batch_size = BATCH_SIZE, 
                 max_token_len = MAX_TOKEN_COUNT,
                 num_workers = 0,
                 pin_memory = False):
        
        super().__init__()

        self.save_hyperparameters(logger=False)
    
    def setup(self, stage=None):

        # choose fold to train on
        kf = StratifiedKFold(n_splits=self.hparams.num_splits, shuffle=True, random_state=self.hparams.split_seed)
        all_splits = [k for k in kf.split(df, df.label)]
        train_indexes, val_indexes = all_splits[self.hparams.k]
        train_indexes, val_indexes = train_indexes.tolist(), val_indexes.tolist()

        self.data_train, self.data_val = df.iloc[train_indexes], df.iloc[val_indexes]
        
        self.train_dataset = SpecterDataset(
          self.data_train,
          self.hparams.tokenizer,
          self.hparams.max_token_len
        )
        self.val_dataset = SpecterDataset(
          self.data_val,
          self.hparams.tokenizer,
          self.hparams.max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
          self.train_dataset,
          batch_size = self.hparams.batch_size,
          shuffle=True
        )
    
    def val_dataloader(self):
        return DataLoader(
          self.val_dataset,
          batch_size = self.hparams.batch_size
        )

### Modelling

In [12]:
class SpecterClassModel(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    
        super().__init__()
        self.specter = AutoModel.from_pretrained(MODEL_NAME, return_dict=True)
        # Changes: Edit model architecture forward pass here
        self.classifier = nn.Linear(self.specter.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.CrossEntropyLoss()
        self.acc = MulticlassAccuracy(num_classes = 21, average = 'weighted') #measure performance based on weighted average
        self.f1 = MulticlassF1Score(num_classes = 21, average = 'weighted')
        self.prec = MulticlassPrecision(num_classes = 21, average = 'weighted')
        self.rec = MulticlassRecall(num_classes = 21, average = 'weighted')
        
        # Changes: Comment below code to remove freezing of the SPECTER embeddings
        for name, param in self.specter.named_parameters():
            if name.startswith('embeddings'):
                param.requires_grad = False
    
    
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.specter(input_ids, attention_mask=attention_mask)
        # Changes: Edit model architecture forward pass here
        output = self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
            acc = self.acc(output, labels)
            f1 = self.f1(output, labels)
            prec = self.prec(output, labels)
            rec = self.rec(output, labels)
        return loss, output, acc, f1, prec, rec

    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, output, acc, f1, prec, rec = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": output, "labels": labels}

    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, output, acc, f1, prec, rec = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": output, "labels": labels, "acc": acc, "f1": f1, "prec": prec, "rec": rec}
    
    def training_epoch_end(self, outputs):
        avg_loss = sum(output['loss'].item() for output in outputs) / len(outputs)
        self.logger.experiment.add_scalars('loss', {'train': avg_loss}, self.current_epoch)   
    
    def validation_epoch_end(self, outputs):
        avg_loss = sum(output['loss'].item() for output in outputs) / len(outputs)
        avg_f1 = sum(output['f1'].item() for output in outputs) / len(outputs)
        avg_prec = sum(output['prec'].item() for output in outputs) / len(outputs)
        avg_rec = sum(output['rec'].item() for output in outputs) / len(outputs)
        avg_acc = sum(output['acc'].item() for output in outputs) / len(outputs)
        
        #For final output/Tensorboard visualisation
        self.log("Ignore/avg_acc", avg_acc, logger=True)
        self.log("Ignore/avg_f1", avg_f1, logger=True)
        self.log("Ignore/avg_prec", avg_prec, logger=True)
        self.log("Ignore/avg_rec", avg_rec, logger=True)
        
        #For Tensorboard visualisaion with Epoch as x axis
        self.logger.experiment.add_scalar('Collated/avg_acc', avg_acc, self.current_epoch)
        self.logger.experiment.add_scalar('Collated/avg_f1', avg_f1, self.current_epoch)
        self.logger.experiment.add_scalar('Collated/avg_prec', avg_prec, self.current_epoch)
        self.logger.experiment.add_scalar('Collated/avg_rec', avg_rec, self.current_epoch)
    
        self.logger.experiment.add_scalars('loss', {'val': avg_loss}, self.current_epoch)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5) # Changes: Edit the learning rate
        scheduler = get_linear_schedule_with_warmup( # Changes: Edit the scheduler
          optimizer,
          num_warmup_steps=self.n_warmup_steps,
          num_training_steps=self.n_training_steps
        )
        
        # Changes: Edit the optimizer
        return dict(
          optimizer = optimizer,
          lr_scheduler = dict(
            scheduler = scheduler,
            interval = 'step'
          )
        )

In [13]:
experiment = "Default"
time_now = time.strftime("%d_%m_%Y_%H_%M")

In [14]:
checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename = f"best-checkpoint-{experiment}",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min"
)

# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [15]:
steps_per_epoch= round(len(df) * 0.8 // BATCH_SIZE)
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(254, 1270)

In [16]:
result_acc_lst = []
result_f1_lst = []
result_prec_lst = []
result_recall_lst = []
nums_fold = 5
split_seed = 123

for k in range(nums_fold):
    data_module = SpecterDataModule(df, tokenizer, k = k)

    # here we train the model on given split...
    logger = TensorBoardLogger("lightning_logs", name = f"Specter_{experiment}_{time_now}_run{k}")
    model = SpecterClassModel(n_classes = 21, n_warmup_steps = warmup_steps, n_training_steps = total_training_steps)
    trainer = pl.Trainer(logger = logger, callbacks = [checkpoint_callback], max_epochs = N_EPOCHS, accelerator = "auto")
    trainer.fit(model, data_module)

    accuracy = trainer.logged_metrics['Collated/avg_acc'].item()
    f1 = trainer.logged_metrics['Collated/avg_f1'].item()
    precision = trainer.logged_metrics['Collated/avg_prec'].item()
    recall = trainer.logged_metrics['Collated/avg_rec'].item()
    
    result_acc_lst.append(accuracy)
    result_f1_lst.append(f1)
    result_prec_lst.append(precision)
    result_recall_lst.append(recall)

average_val_acc_score = sum(result_acc_lst) / len(result_acc_lst)
average_val_f1_score = sum(result_f1_lst) / len(result_f1_lst)
average_val_prec_score = sum(result_prec_lst) / len(result_prec_lst)
average_val_recall_score = sum(result_recall_lst) / len(result_recall_lst)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs\Specter_Default_30_01_2023_11_13_run0
C:\Users\Benjamin Aw\anaconda3\envs\myenv\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:604: UserWarning: Checkpoint directory C:\Users\Benjamin Aw\Desktop\Extract Scientific Papers\Experiments\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\Benjamin Aw\anaconda3\envs\myenv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name       | Type                | Params
--------------------------

Sanity Checking DataLoader 0:   0%|                                                              | 0/2 [00:00<?, ?it/s]

C:\Users\Benjamin Aw\anaconda3\envs\myenv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Users\Benjamin Aw\anaconda3\envs\myenv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  80%|██████████████████████████▍      | 255/319 [00:52<00:13,  4.84it/s, loss=1.89, v_num=0, train_loss=1.660]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|█████████████████| 319/319 [00:58<00:00,  5.46it/s, loss=1.89, v_num=0, train_loss=1.660, val_loss=1.670]

...

Epoch 2:  80%|████████████▊   | 255/319 [00:53<00:13,  4.80it/s, loss=0.833, v_num=0, train_loss=0.666, val_loss=1.310]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|████████████████| 319/319 [00:58<00:00,  5.45it/s, loss=0.833, v_num=0, train_loss=0.666, val_loss=1.270]

Epoch 2, global step 765: 'val_loss' was not in top 1


Epoch 3:  80%|████████████▊   | 255/319 [00:53<00:13,  4.80it/s, loss=0.687, v_num=0, train_loss=1.270, val_loss=1.270]
Validation: 0it [00:00, ?it/s]
Epoch 4:  80%|████████████▊   | 255/319 [00:53<00:13,  4.80it/s, loss=0.576, v_num=0, train_loss=0.521, val_loss=1.250]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|████████████████| 319/319 [00:58<00:00,  5.44it/s, loss=0.576, v_num=0, train_loss=0.521, val_loss=1.270]

Epoch 4, global step 1275: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|████████████████| 319/319 [00:58<00:00,  5.44it/s, loss=0.576, v_num=0, train_loss=0.521, val_loss=1.270]


In [17]:
print(f"The average accuracy of the validation set across 5 folds is: {average_val_acc_score}")
print(f"The average F1 score of the validation set across 5 folds is: {average_val_f1_score}")
print(f"The average precision of the validation set across 5 folds is: {average_val_prec_score}")
print(f"The average recall of the validation set across 5 folds is: {average_val_recall_score}")

The average accuracy of the validation set across 5 folds is: 0.6642968654632568
The average F1 score of the validation set across 5 folds is: 0.7212699890136719
The average precision of the validation set across 5 folds is: 0.8483537912368775
The average recall of the validation set across 5 folds is: 0.6642968654632568


### Stop here for now We might want to take a look at creating a test set 